In [1]:
import duckdb
import os
from dotenv import load_dotenv
import duckdb

load_dotenv()  # loads .env into environment

db_path = os.getenv("DUCKDB_PATH", "data.duckdb")  # default fallback

con = duckdb.connect(db_path)  # this creates/opens the file

# Create a table if it doesn't exist

con.execute(
    """
CREATE TABLE IF NOT EXISTS platform_metrics (
    car_id       TEXT,
    platform     TEXT,
    run_ts       TIMESTAMP,
    metrics      JSON,
    PRIMARY KEY (car_id, platform, run_ts)
);

CREATE TABLE IF NOT EXISTS overall_cache (
    car_id       TEXT,
    run_ts       TIMESTAMP,
    metrics      JSON,
    PRIMARY KEY (car_id, run_ts)
);
"""
)

In [7]:
from datetime import datetime
from pydantic import BaseModel
import json

# Sample Pydantic model


class Metrics(BaseModel):
    num_comments: int
    avg_sentiment_score: float
    most_common_sentiment: str
    likes: int
    shares: int
    plays: int
    collections: int
    engagement_score: float
    overall_sentiment_score: float


m = Metrics(
    num_comments=42,
    avg_sentiment_score=0.75,
    most_common_sentiment="positive",
    likes=100,
    shares=8,
    plays=500,
    collections=5,
    engagement_score=0.85,
    overall_sentiment_score=0.78,
)

con.execute(
    """
    INSERT INTO platform_metrics (car_id, platform, run_ts, metrics)
    VALUES (?, ?, ?, ?)
""",
    ["car123", "tiktok", datetime.now(), m.model_dump_json()],
)

In [8]:
rows = con.execute(
    """
SELECT platform, metrics
FROM (
  SELECT *, row_number() OVER (PARTITION BY platform ORDER BY run_ts DESC) AS rn
  FROM platform_metrics
  WHERE car_id = ?
)
WHERE rn = 1
""",
    ["car123"],
).fetchall()

for platform, metrics_blob in rows:
    metrics = json.loads(metrics_blob)
    print(platform, metrics)

tiktok {'num_comments': 42, 'avg_sentiment_score': 0.75, 'most_common_sentiment': 'positive', 'likes': 100, 'shares': 8, 'plays': 500, 'collections': 5, 'engagement_score': 0.85, 'overall_sentiment_score': 0.78}
